In [1]:
import numpy as np
import pandas as pd

import glob

import warnings
warnings.filterwarnings('ignore')

In [2]:
paths = glob.glob('../Data/*.csv')
df = []
for i in paths:
    df.append(pd.read_csv(i))

data = df[0]
for i in range(1, len(paths)):
    data = pd.concat((data, df[i]), ignore_index = True)

In [3]:
data.head()

,category,description,details,discount,is_fassured,mrp,num_ratings,num_ratings_reviews,price,rating,timestamp
0,Air Conditioner,Midea 1 Ton 3 Star Split AC - White,Condenser Coil: CopperPower Consumption: 980 W...,48% off,1,"₹39,600",NaN,"15,595 Ratings & 2,685 Reviews","₹20,499",4.2,2019-08-14 23:49:25.955540
1,Air Conditioner,Voltas 1.5 Ton 3 Star Split AC - White,Condenser Coil: CopperPower Consumption: 1450 ...,45% off,1,"₹51,990",NaN,"5,325 Ratings & 549 Reviews","₹28,499",4.2,2019-08-14 23:49:25.956537
2,Air Conditioner,MarQ by Flipkart 1.5 Ton 3 Star Split Inverter...,Condenser Coil: CopperPower Consumption: 1650 ...,46% off,1,"₹46,999",NaN,"10,058 Ratings & 1,676 Reviews","₹24,999",4.2,2019-08-14 23:49:25.957534
3,Air Conditioner,Voltas 1.5 Ton 3 Star Split Inverter AC - White,Condenser Coil: CopperPower Consumption: 1750 ...,42% off,1,"₹55,990",NaN,"4,423 Ratings & 453 Reviews","₹31,999",4.2,2019-08-14 23:49:25.959529
4,Air Conditioner,Voltas 1.2 Ton 5 Star Split Inverter AC - White,Condenser Coil: CopperPower Consumption: 1010 ...,43% off,1,"₹54,990",NaN,"3,146 Ratings & 478 Reviews","₹30,999",4.1,2019-08-14 23:49:25.960526


In [4]:
data.tail()

,category,description,details,discount,is_fassured,mrp,num_ratings,num_ratings_reviews,price,rating,timestamp
25957,Wearable Smart Devices,Gazzet 4G V9 green Calling watch Android Smart...,NaN,58% off,1,"₹5,999",(13),NaN,"₹2,499",4.0,2019-08-14 22:27:22.886805
25958,Wearable Smart Devices,Everycom Replacement Premium Hex Design TPU St...,NaN,42% off,0,₹699,(12),NaN,₹399,3.3,2019-08-14 22:27:22.887803
25959,Wearable Smart Devices,TRENDONIX GT08-43 phone Smartwatch,NaN,84% off,0,"₹4,999",0,NaN,₹799,0.0,2019-08-14 22:27:22.887803
25960,Wearable Smart Devices,MStick Replacement TPU Strap For Xiaomi Mi Ban...,NaN,69% off,0,₹549,(116),NaN,₹165,3.9,2019-08-14 22:27:22.888800
25961,Wearable Smart Devices,ACUTAS Replacement Sports Ventilation Hole Str...,NaN,56% off,1,"₹1,499",(6),NaN,₹649,4.0,2019-08-14 22:27:22.888800


In [5]:
brand_names = pd.read_csv('../Data/Brands/Brands_Data.csv')
brand_names.head()

,brand_names,timestamp
0,#CELL,2019-08-19 12:57:18.281336
1,#TAG,2019-08-19 12:57:18.281336
2,007,2019-08-19 12:57:18.281336
3,1 Art Of Creations,2019-08-19 12:57:18.281336
4,10 Case,2019-08-19 12:57:18.281336


In [6]:
brand_names.isnull().sum()

brand_names    0
timestamp      0
dtype: int64

In [7]:
# Checking null values
data.isnull().sum()

category                   0
description                0
details                20501
discount                   0
is_fassured                0
mrp                        0
num_ratings             5461
num_ratings_reviews    20501
price                      0
rating                     0
timestamp                  0
dtype: int64

In [8]:
# Dtypes
data.dtypes

category                object
description             object
details                 object
discount                object
is_fassured              int64
mrp                     object
num_ratings             object
num_ratings_reviews     object
price                   object
rating                 float64
timestamp               object
dtype: object

In [9]:
# Creating copy to clean the dataframe
data1 = data.copy()

In [10]:
import re

In [11]:
# Fetching numbers from price and mrp columns
def fetch_nos(x):
    y = re.findall('\d', x)
    return float(''.join(y))

data1['price'] = data1['price'].apply(fetch_nos)
data1['mrp'] = data1['mrp'].apply(fetch_nos)

In [12]:
data1[['price', 'mrp']].head()

,price,mrp
0,20499.0,39600.0
1,28499.0,51990.0
2,24999.0,46999.0
3,31999.0,55990.0
4,30999.0,54990.0


In [13]:
data1['discount'].value_counts()

0          1775
50% off    1011
70% off     712
60% off     614
73% off     536
           ... 
90% off       1
₹2 off        1
₹4 off        1
₹3 off        1
₹13 off       1
Name: discount, Length: 95, dtype: int64

In discount column, there are entries as 0 which means no discount. Apart from this percentage discount is there and discount in ruppee is also there. <br>
We convert all the value as percentage discount.

In [14]:
# Fetching percentage of discount from discount column
def percent_discount(x):
    if x[2].strip() == '0' or '%' in x[2]:
        y = re.findall('\d', x[2])
        return float(''.join(y))
    else:
        return (round((x[0] - x[1])/x[0], 2))

data1['discount (%)'] = data1[['mrp', 'price', 'discount']].apply(percent_discount, axis = 1)

In [15]:
data1['discount (%)'].value_counts()

0.00     1775
50.00    1011
70.00     712
60.00     614
73.00     536
         ... 
0.04        3
0.02        1
0.03        1
0.07        1
90.00       1
Name: discount (%), Length: 94, dtype: int64

In [16]:
data1[['discount', 'discount (%)']].isnull().sum()

discount        0
discount (%)    0
dtype: int64

In [17]:
# Filling null values in 'num_ratings' column with number of ratings from 'num_ratings_reviews' column

def ratings(x):
    if pd.isnull(x[0]):
        rt = re.findall('\d', x[1].split('&')[0])
        return float(''.join(rt))
    else:
        rt = re.findall('\d', x[0])
        return float(''.join(rt))

data1['num_ratings'] = data1[['num_ratings', 'num_ratings_reviews']].apply(ratings, axis = 1)

In [18]:
data1['num_ratings'].iloc[:5]

0    15595.0
1     5325.0
2    10058.0
3     4423.0
4     3146.0
Name: num_ratings, dtype: float64

In [19]:
data1['num_ratings'].iloc[-1: -6: -1]

25961      6.0
25960    116.0
25959      0.0
25958     12.0
25957     13.0
Name: num_ratings, dtype: float64

In [20]:
data1['num_ratings'].isnull().sum()

0

'num_values' column has been taken care of and there is no null values now. Also, from the first and last five values in 'num_rating' columns, we can see that transformation has been done correctly.

In [21]:
data1['num_ratings_reviews'].value_counts()

0                              1606
4 Ratings & 0 Reviews            58
3 Ratings & 0 Reviews            57
7 Ratings & 0 Reviews            57
8 Ratings & 0 Reviews            51
                               ... 
2,778 Ratings & 447 Reviews       1
257 Ratings & 40 Reviews          1
499 Ratings & 112 Reviews         1
87 Ratings & 4 Reviews            1
4,176 Ratings & 925 Reviews       1
Name: num_ratings_reviews, Length: 1861, dtype: int64

In [22]:
# Creating a new column as 'num_reviews' to capture number of reviews on a product from 'num_ratings_reviews' column
# Note: 0 in 'num_ratings_reviews' column means there is no rating and no review for that product

def reviews(x):
    if pd.notnull(x):
        rw = re.findall('\d', x.split('&')[-1])   # It will handle conditions for both one and two element list
        return float(''.join(rw))
    else:
        return x
    
data1['num_reviews'] = data1['num_ratings_reviews'].apply(reviews)

In [23]:
data1['num_reviews'].iloc[:5]

0    2685.0
1     549.0
2    1676.0
3     453.0
4     478.0
Name: num_reviews, dtype: float64

In [24]:
data1['num_reviews'].iloc[-1 : -6 : -1]

25961   NaN
25960   NaN
25959   NaN
25958   NaN
25957   NaN
Name: num_reviews, dtype: float64

In [25]:
data1['num_reviews'].isnull().sum() == data1['num_ratings_reviews'].isnull().sum()

True

'num_reviews' column has been correctly created is has null values, means reviews for that products are not captured during scraping.  

In [47]:
keywords = []
for i in brand_names['brand_names']:
     keywords.append(i.strip().split())

In [48]:
# Fetching brand from description where brand matches brand_names

def fetch_brand(x):
    words = x.lower().strip().split()[:2]
    brand, y = '', []
    for i in keywords:
        k = [a.lower() for a in i]
        if ((words[0].lower() in k) & (words[1].lower() in k)):
            brand = ' '.join(i)
            break
        
    else:
        brand = x.strip().split()[0]
        
    return brand

In [49]:
# Will take some time
data1['brand'] = data1['description'].apply(fetch_brand)

In [59]:
data1.dtypes

category                object
description             object
details                 object
discount                object
is_fassured              int64
mrp                    float64
num_ratings            float64
num_ratings_reviews     object
price                  float64
rating                 float64
timestamp               object
discount (%)           float64
num_reviews            float64
brand                   object
dtype: object

In [60]:
# Exporting cleaned dataframe
data1[
    ['category', 'brand', 'description', 'details', 'is_fassured', 'rating', 'discount (%)', 'price', 'mrp', 'num_ratings',
    'num_reviews', 'timestamp']
].to_csv('../Data/Cleaned_Data/Cleaned_Data.csv', index = False)